In [2]:
import aiohttp
import asyncio
from os.path import exists
import pandas as pd
folder = '/Users/muneebalam/Desktop/Imperial/ML/project/scraped-emails/emails/'
LIMIT = 30322
req_lim = 100
urlbase = 'https://www.wikileaks.org/clinton-emails/emailid/'

In [3]:
@asyncio.coroutine
def get(*args, **kwargs):
    response = yield from aiohttp.request('GET', *args, **kwargs)
    return (yield from response.text())

@asyncio.coroutine
def extract_text(url, sem):
    with (yield from sem):
        page = yield from get(url)
        w = open(get_save_file(url=url), 'w')
        w.write(page)
        w.close()
        emailid = int(url[url.rfind('/')+1:])
        if emailid % 1000 == 0:
            print('Writing', emailid)

def generate_links(start):
    return [urlbase+str(x) for x in range(start, LIMIT+1) if not exists(get_save_file(x))]

def get_save_file(emailid=None, url=None):
    if emailid is None:
        emailid = int(url[url.rfind('/')+1:])
    return folder + str(emailid) + '.txt'

@asyncio.coroutine
def run(links):
    sem = asyncio.Semaphore(req_lim)
    fetchers = [extract_text(link, sem) for link in links]
    return [(yield from f) for f in asyncio.as_completed(fetchers)]

In [58]:
loop = asyncio.get_event_loop()
x = loop.run_until_complete(run(generate_links(1)))

In [16]:
from datetime import datetime
def parse_file(emailid):
    r = open(get_save_file(emailid))
    data = r.read()
    r.close()
    
    sent = 'n/a'
    released = 'n/a'
    sender = 'n/a'
    recipient = 'n/a'
    text = 'n/a'
    subject = 'n/a'
    
    data = data[data.index('Share document'):]
    
    if 'Original Message' in data:
        data = data[:data.index('Original Message')]
    else:
        data = data[:data.index('</div>')]
    
    i = data.index('From:')
    data = data[i:]
    sender = data[data.index('>')+1:data.index('</span>')].strip()
    #print('From:\t', sender)
    
    i = data.index('To:')
    data = data[i:]
    recipient = data[data.index('>')+1:data.index('</span>')].strip()
    #print('To:\t', recipient)
    
    i = data.index('Date:')
    data = data[i:]
    sent = data[5:data.index('Subject')].strip()
    j = sent.index(' ')
    sent, time = sent.split(' ')[:2]
    hr, minute = [int(x) for x in time.split(':')]
    y, m, d = [int(x.strip()) for x in sent.split('-')]
    sent = datetime(y, m, d, hr, minute)
    #print('Sent:\t', sent)
    
    i = data.index('Subject:')
    data = data[i:]
    subject = data[8:data.index('</header>')].strip()
    #print('Subject:\t', subject)
    
    if emailid == 7632:
        i = data.index('D ate:')
    elif emailid == 9700:
        i = data.index('D6a t e') + 2
    elif emailid == 14704:
        i = data.index('Date.')
    elif emailid == 23805:
        i = data.index('D a te :') + 3
    else:
        i = data.index('Date:')
    data = data[i:]
    released = data[5:data.index('</span>')].strip().replace(' ', '')
    if released[-1] == '-':
        released = released[:-1]
    if released[0] == ':':
        released = released[1:]
    if len(released) > 10:
        released = released[:10]
    m, d, y = [int(x.strip()) for x in released.split('/')]
    released = datetime(y, m, d)
    #print('Released:\t', released)
    
    data = data.split('</span>')[-1]
    text = data.replace('<span class="inlinemeta">', '').strip()
    #print('Text:\t', text)
    
    return sent, released, sender, recipient, subject, text

In [17]:
data = []
skip = {15406}
for i in range(LIMIT):
    if (i+1) not in skip:
        try:
            data.append(parse_file(i+1))
        except Exception as e:
            print(i+1, e, e.args)
            break
    if (i+1) % 10000 == 0:
        print('Done extracting through', i+1)
df = pd.DataFrame.from_records(data)
df.columns = ['Sent', 'Released', 'Sender', 'Recipient', 'Subject', 'Text']

Done extracting through 10000
Done extracting through 20000
Done extracting through 30000


In [18]:
def read_nrc():
    #returns {word: (positive/negative, {emotions})}
    r = open('/Users/muneebalam/Desktop/Imperial/ML/project/NRC-Emotion-Lexicon/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt')
    data = r.read().strip()
    r.close()
    
    firstword = 'aback'
    data = data[data.index(firstword):].split('\n')
    lex = {}
    generalset = {'positive', 'negative'}
    for line in data:
        word, emot, posneg = line.split('\t')
        posneg = int(posneg)
        if word not in lex:
            lex[word] = [None, set()]
        if posneg == 1:
            if emot in generalset:
                lex[word][0] = emot
            else:
                lex[word][1].add(emot)
    return lex
sent_dct = read_nrc()

In [19]:
from nltk.corpus import stopwords
stop_word_lst = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize
opposites = {'anger': 'fear', 'anticipation': 'surprise', 'disgust': 'trust', 'fear': 'anger', 
             'joy': 'sadness', 'sadness': 'joy', 'surprise': 'anticipation', 'trust': 'disgust',
            'positive': 'negative', 'negative': 'positive'}
negation_words = {'not'}
def split_by_words(text):
    return tokens = word_tokenize(raw)
def findemot(text):
    emots = {'anger': 0, 'anticipation': 0, 'disgust': 0, 'fear': 0,
        'joy': 0, 'sadness': 0, 'surprise': 0, 'trust': 0}
    posnegneu = {'positive': 0, 'negative': 0, 'neutral': 0}
    wordcount = 0
    #https://en.wikipedia.org/wiki/Contrasting_and_categorization_of_emotions#Plutchik.27s_wheel_of_emotions
    #emotions have opposites
    try:
        sentences = split_by_sentences(text)
        if len(sentences) == 1:
            negation = False
            words = split_by_words(text)
            for word in words:
                wordcount += 1
                if word in negation_words:
                    negation = not negation
                if word in stop_word_lst:
                    pass
                else:
                    if word in sent_dct:
                        if sent_dct[word][0] is None:
                            posnegneu['neutral'] += 1
                        else:
                            if negation:
                                posnegneu[opposites[sent_dct[word][0]]] += 1
                            else:
                                posnegneu[sent_dct[word][0]] += 1
                        for em in sent_dct[word][1]:
                            if negation:
                                emots[opposites[em]] += 1
                            else:
                                emots[em] += 1
                    else:
                        word2 = SnowballStemmer("english").stem(word)
                        if word2 in sent_dct:
                            if sent_dct[word2][0] is None:
                                posnegneu['neutral'] += 1
                            else:
                                if negation:
                                    posnegneu[opposites[sent_dct[word2][0]]] += 1
                                else:
                                    posnegneu[sent_dct[word2][0]] += 1
                            for em in sent_dct[word2][1]:
                                if negation:
                                    emots[opposites[em]] += 1
                                else:
                                    emots[em] += 1
        else:
            sents = [findemot(sen) for sen in sentences]
            for sen_pnn, sen_emot, wc in sents:
                wordcount += wc
                for sen in sen_pnn:
                    posnegneu[sen] += sen_pnn[sen]
                for sen in sen_emot:
                    emots[sen] += sen_emot[sen]
    except AttributeError:
        pass
    return posnegneu, emots, wordcount
def extract_anger(dcts):
    return dcts[1]['anger']
def extract_anticipation(dcts):
    return dcts[1]['anticipation']
def extract_disgust(dcts):
    return dcts[1]['disgust']
def extract_fear(dcts):
    return dcts[1]['fear']
def extract_joy(dcts):
    return dcts[1]['joy']
def extract_sadness(dcts):
    return dcts[1]['sadness']
def extract_surprise(dcts):
    return dcts[1]['surprise']
def extract_trust(dcts):
    return dcts[1]['trust']
def extract_positive(dcts):
    return dcts[0]['positive']
def extract_negative(dcts):
    return dcts[0]['negative']
def extract_neutral(dcts):
    return dcts[0]['neutral']
def extract_wordcount(dcts):
    return dcts[2]

In [20]:
emots = {'anger': extract_anger, 'anticipation': extract_anticipation, 
         'disgust': extract_disgust, 'fear': extract_fear,
        'joy': extract_joy, 'sadness': extract_sadness, 
         'surprise': extract_surprise, 'trust': extract_trust}
posnegneu = {'positive': extract_positive, 'negative': extract_negative, 
             'neutral': extract_neutral}
df['sentinfo'] = df['Text'].apply(findemot) 
for e in emots:
    df[e] = df['sentinfo'].apply(emots[e])
for e in posnegneu:
    df[e] = df['sentinfo'].apply(posnegneu[e])
df['wordcount'] = df['sentinfo'].apply(extract_wordcount)

In [21]:
df.drop('sentinfo', axis=1, inplace=True)
df.to_csv(folder + 'wikiemails.csv')

In [104]:
def resentiment():
    df = pd.read_csv(folder + 'wikiemails.csv')
    emots = {'anger': extract_anger, 'anticipation': extract_anticipation, 
         'disgust': extract_disgust, 'fear': extract_fear,
        'joy': extract_joy, 'sadness': extract_sadness, 
         'surprise': extract_surprise, 'trust': extract_trust}
    posnegneu = {'positive': extract_positive, 'negative': extract_negative, 
                 'neutral': extract_neutral}
    df['sentinfo'] = df['Text'].apply(findemot) 
    for e in emots:
        df[e] = df['sentinfo'].apply(emots[e])
    for e in posnegneu:
        df[e] = df['sentinfo'].apply(posnegneu[e])
    df['wordcount'] = df['sentinfo'].apply(extract_wordcount)

In [15]:
for i in range(1, 10):
    parse_file(i)

06:53
Sent:	 2010-06-30 06:53:00
00:57
Sent:	 2010-06-17 00:57:00
19:30
Sent:	 2010-06-15 19:30:00
10:09
Sent:	 2010-06-27 10:09:00
06:25
Sent:	 2010-06-30 06:25:00
20:37
Sent:	 2010-06-15 20:37:00
01:30
Sent:	 2010-06-24 01:30:00
00:38
Sent:	 2010-07-01 00:38:00
01:13
Sent:	 2010-06-19 01:13:00
